# 01. Ollama + Qwen3 기초

## 🎯 학습 목표
1. Ollama의 개념과 작동 원리 이해
2. Qwen3 모델의 특징과 장점 파악
3. 로컬 LLM 실행 및 활용법 습득
4. Thinking Mode를 통한 심층 추론 활용

## 📚 핵심 개념

## 1. Ollama란?

**Ollama**는 대규모 언어 모델(LLM)을 로컬에서 쉽게 실행할 수 있게 해주는 도구입니다.

### 주요 특징:
- 🖥️ **로컬 실행**: 인터넷 연결 없이 개인 컴퓨터에서 실행
- 🔒 **프라이버시**: 데이터가 외부로 전송되지 않음
- 💰 **무료**: API 비용 없이 무제한 사용
- 🚀 **간편한 설정**: 한 줄 명령으로 모델 다운로드 및 실행

### 작동 원리:
```
사용자 → Ollama CLI → 모델 다운로드 → 로컬 서버 실행 → REST API 제공
```

## 2. Qwen3 모델 소개

**Qwen3**는 Alibaba에서 2025년 4월에 출시한 최신 언어 모델입니다.

### Qwen3 vs Qwen2 비교:

| 특징 | Qwen2 | Qwen3 |
|------|-------|-------|
| 추론 능력 | 좋음 | 매우 뛰어남 (QwQ 수준) |
| 모델 효율성 | 일반 | 뛰어남 (4B가 72B 성능) |
| Thinking Mode | ❌ | ✅ /think, /no_think |
| MoE 지원 | 제한적 | 완전 지원 |
| 코딩 능력 | 좋음 | 매우 뛰어남 |

### Thinking Mode란?
- 복잡한 문제에 대해 단계별로 사고하는 모드
- `/think`로 활성화, `/no_think`로 비활성화
- Chain of Thought (CoT) 추론을 자동으로 수행

## 3. 설치 및 환경 설정

In [ ]:
# Ollama 설치 확인
!ollama --version

In [ ]:
# Qwen3 모델 다운로드 (처음 한 번만)
# 크기별 선택:
# - qwen3:4b (작지만 강력)
# - qwen3:8b (균형잡힌 선택) 
# - qwen3:30b-a3b (MoE, 고성능)

!ollama pull qwen3:8b

In [ ]:
# 설치된 모델 확인
!ollama list

## 4. 기본 사용법

In [ ]:
import requests
import json

def chat_with_qwen(prompt, model="qwen3:8b", thinking_mode=False):
    """Qwen3와 대화하는 함수"""
    
    # Thinking Mode 적용
    if thinking_mode:
        prompt = f"/think {prompt}"
    
    # API 호출
    response = requests.post(
        'http://localhost:11434/api/generate',
        json={
            "model": model,
            "prompt": prompt,
            "stream": False
        }
    )
    
    if response.status_code == 200:
        return response.json()['response']
    else:
        return f"Error: {response.status_code}"

In [9]:
# 간단한 질문
response = chat_with_qwen("파이썬의 장점을 3가지 알려주세요")
print(response)

<think>
Okay, the user is asking for three strengths of Python. Let me start by recalling what I know about Python's advantages.

First, Python is known for its simplicity and readability. The syntax is straightforward, which makes it easy for beginners to learn and for experienced developers to write code that's easy to maintain. That's a solid point.

Second, Python has a vast standard library and a rich ecosystem of third-party packages. This means you can find libraries for almost any task, from web development to data analysis, which saves time and effort. For example, libraries like NumPy, Pandas, and Django are really popular.

Third, Python is versatile and can be used across different domains. Whether it's web development, scientific computing, automation, or machine learning, Python has the tools and frameworks to handle it. This versatility makes it a go-to language for many developers.

Wait, are there other strengths? Maybe things like dynamic typing or integration with ot

## 5. Thinking Mode 활용

복잡한 추론이 필요한 문제에서 Thinking Mode를 활용해봅시다.

In [ ]:
# 일반 모드 vs Thinking Mode 비교
problem = "농부가 강을 건너야 하는데, 늑대, 양, 양배추를 가지고 있습니다. 보트는 농부와 한 가지만 실을 수 있습니다. 늑대와 양을 혼자 두면 늑대가 양을 먹고, 양과 양배추를 혼자 두면 양이 양배추를 먹습니다. 어떻게 모두 안전하게 강을 건널 수 있을까요?"

print("=== 일반 모드 ===")
normal_response = chat_with_qwen(problem, thinking_mode=False)
print(normal_response[:500] + "...\n")

print("=== Thinking Mode ===")
thinking_response = chat_with_qwen(problem, thinking_mode=True)
print(thinking_response[:500] + "...")

## 6. 스트리밍 응답

실시간으로 응답을 받아보는 스트리밍 모드를 구현해봅시다.

In [ ]:
def stream_chat(prompt, model="qwen3:8b"):
    """스트리밍 방식으로 응답 받기"""
    
    response = requests.post(
        'http://localhost:11434/api/generate',
        json={
            "model": model,
            "prompt": prompt,
            "stream": True
        },
        stream=True
    )
    
    full_response = ""
    for line in response.iter_lines():
        if line:
            data = json.loads(line)
            token = data.get('response', '')
            print(token, end='', flush=True)
            full_response += token
            
            if data.get('done', False):
                break
    
    return full_response

In [ ]:
# 스트리밍으로 이야기 생성
story = stream_chat("인공지능이 주인공인 짧은 SF 이야기를 만들어주세요 (100자 이내)")

## 7. 실전 활용 예제

In [ ]:
class QwenAssistant:
    """실무에서 사용할 수 있는 Qwen3 어시스턴트"""
    
    def __init__(self, model="qwen3:8b"):
        self.model = model
        self.base_url = "http://localhost:11434"
    
    def code_review(self, code):
        """코드 리뷰"""
        prompt = f"""
        다음 코드를 리뷰해주세요:
        
        ```python
        {code}
        ```
        
        1. 잠재적 버그
        2. 성능 개선점
        3. 코드 스타일 제안
        """
        return chat_with_qwen(prompt, self.model)
    
    def explain_error(self, error_message):
        """에러 메시지 설명"""
        prompt = f"""
        다음 에러를 설명하고 해결 방법을 알려주세요:
        
        {error_message}
        """
        return chat_with_qwen(prompt, self.model)
    
    def generate_docstring(self, function_code):
        """함수 문서화 자동 생성"""
        prompt = f"""
        다음 함수에 대한 docstring을 작성해주세요:
        
        {function_code}
        
        Google 스타일 docstring으로 작성해주세요.
        """
        return chat_with_qwen(prompt, self.model)
    
    def sql_from_text(self, description, schema=None):
        """자연어를 SQL로 변환"""
        prompt = f"자연어: {description}\n"
        if schema:
            prompt += f"스키마: {schema}\n"
        prompt += "SQL 쿼리:"
        return chat_with_qwen(prompt, self.model, thinking_mode=True)

In [ ]:
# 어시스턴트 사용 예제
assistant = QwenAssistant()

# 1. 코드 리뷰
sample_code = """
def calculate_average(numbers):
    sum = 0
    for i in range(len(numbers)):
        sum = sum + numbers[i]
    average = sum / len(numbers)
    return average
"""

print("=== 코드 리뷰 ===")
review = assistant.code_review(sample_code)
print(review)

In [ ]:
# 2. SQL 생성
print("\n=== SQL 생성 ===")
sql = assistant.sql_from_text(
    "지난 달 매출이 100만원 이상인 고객들의 이름과 이메일을 찾아줘",
    schema="customers(id, name, email), orders(id, customer_id, amount, date)"
)
print(sql)

## 8. 성능 최적화 팁

### 메모리 관리
```bash
# GPU 메모리 제한
export OLLAMA_MAX_LOADED_MODELS=1
export OLLAMA_NUM_GPU=1

# CPU 스레드 설정
export OLLAMA_NUM_THREAD=8
```

### 모델 선택 가이드
- **qwen3:4b**: 빠른 응답이 필요한 경우 (채팅봇, 실시간 처리)
- **qwen3:8b**: 균형잡힌 선택 (일반적인 작업)
- **qwen3:30b-a3b**: 복잡한 추론, 코드 생성 (MoE 아키텍처)

### 프롬프트 최적화
1. 명확하고 구체적인 지시
2. 예제 제공 (Few-shot learning)
3. 출력 형식 명시
4. Thinking Mode는 복잡한 문제에만 사용

## 🎯 실습 과제

1. **기본 과제**: 자신만의 프롬프트 3개를 작성하고 일반 모드와 Thinking Mode 비교
2. **심화 과제**: QwenAssistant 클래스에 새로운 메서드 3개 추가
3. **프로젝트**: 특정 도메인(예: 요리, 운동, 공부)에 특화된 어시스턴트 만들기

## 📚 추가 학습 자료

- [Ollama 공식 문서](https://github.com/ollama/ollama)
- [Qwen3 논문](https://qwenlm.github.io/blog/qwen3/)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)

## 다음 단계

다음 노트북에서는 **LangChain**을 활용하여 더 복잡한 체인과 워크플로우를 구성하는 방법을 배워보겠습니다.